In [7]:
%load_ext autoreload
%autoreload 2

### Run MAF on Multiple FBS_1.4 OpSims

The previous example showed how to run a metric analysis on one or two opSims.  But what you really want to do is to be able to run through *all* of them.  This notebook shows how to do that.  The next notebook will demonstrate how to analyze the results.

### **Important:**  
As in the last notebook, in the next cell you need to update the `your_username` variable with **Your Username** (between the single quotes).  After you have done that, in principle, you should be able to run the notebook all at once instead of cell by cell. 

In [1]:
#Please enter your SciServer username between the single quotes below!
your_username = ''

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

Import the sims_maf modules needed.

In [3]:
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [17]:
# import convenience functions
from opsimUtils import *

#### 1. Build connections to the OpSims databases
The first step is to initiate opsim database objects and result database objects for the opsim databases that you want to run metrics on. Two paths are needed here:
1. `dbDir`: The path to the OpSim database directory (OpSims generated by different versions of the FBS are stored seperately)
2. `outDir`: The path to the directory where you want to save the metric metadata.

By providing these two paths, the function `connect_dbs()` will initiate connections and store the metadata to the designated directory. **NOTE:** Opsims simulated using different versions of the Feature-based schedulers are stored in seperate folders, you can get the correct path using function `show_fbs_dirs()`.

In [20]:
# Check avaiable database directoies
show_fbs_dirs()

['/home/idies/workspace/lsst_cadence/FBS_1.4/',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/',
 '/home/idies/workspace/lsst_cadence/FBS_1.4.1/']

In [7]:
if your_username == '': # do NOT put your username here, put it in the cell at the top of the notebook.
    raise Exception('Please provide your username!  See the top of the notebook.')

dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.4/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [8]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

You can use `help` to get more information about the provided convenience functions, e.g., `connect_dbs()`

In [29]:
help(connect_dbs)

Help on function connect_dbs in module opsimUtils:

connect_dbs(dbDir, outDir)
    Initiate database objects to all opSim databases in the provided directory.
    Returns a dictionary consisting all database connections and a dictionary holding
    the resultsDb objects.
    
    Args:
        dbDir(str): The path to the dabase directory.
        outDir(str): The path to the result database directory.
    
    Returns:
        opSimDbs(dict): A dictionary containing the OpsimDatabase objects for opsim databases in the provided directory, keys are the run names.
        resultDbs(str): A dictionary containing the ResultsDb objects for opsim databases in the provided directory, keys are the run names.



You can also check what OpSims are available in the directory. We suggest saving the returned list of the `show_opsims` to a variable, which will make your life easier if you don't want to run metric evaluations on all avaiable opsims. 

In [10]:
dbRuns = show_opsims(dbDir)
dbRuns[0:5] # only show first 5 opsims

['wfd_depth_scale0.85_noddf_v1.4_10yrs',
 'twilight_neo_mod2_v1.4_10yrs',
 'weather_0.3_v1.4_10yrs',
 'pair_strategy_4_v1.4_10yrs',
 'short_exp_2ns_1expt_v1.4_10yrs']

#### 2. Declare some metrics to run on above OpSims

In [11]:
# Airmass Metric
maxMetric = metrics.MaxMetric('airmass')
airmassSlicer = slicers.HealpixSlicer(nside=64)
airmassConstraint = 'filter = "g"'
airmassConstraint += ' and note not like "DD%"'

maxairmassSky = metricBundles.MetricBundle(maxMetric, airmassSlicer, \
                airmassConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [12]:
# Coadd Metric in Y band
coaddMetric = metrics.Coaddm5Metric()
coaddSlicer = slicers.HealpixSlicer(nside=64)
coaddConstraint = 'filter = "y"'
coaddConstraint += ' and note not like "DD%"'

CoaddY = metricBundles.MetricBundle(coaddMetric, coaddSlicer, \
                coaddConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [13]:
# Coadd Metric in U band
coaddMetric = metrics.Coaddm5Metric()
coaddSlicer = slicers.HealpixSlicer(nside=64)
coaddConstraintU = 'filter = "u"'
coaddConstraintU += ' and note not like "DD%"'

CoaddU = metricBundles.MetricBundle(coaddMetric, coaddSlicer, \
                coaddConstraintU)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [14]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
maxairmassSky.setSummaryMetrics(summaryMetrics)
CoaddY.setSummaryMetrics(summaryMetrics)
CoaddU.setSummaryMetrics(summaryMetrics)

In [15]:
# create a bundleDict for all metrics to run on each opSim
bundleDict = {'maxairmassSky': maxairmassSky, 'CoaddY': CoaddY, 'CoaddU': CoaddU}

#### 3. Loop over all OpSims in dbDir and run MAF
While constructing a metricBundleGroup from a dictionary (the cell below), you will need to provide the path to a directory (`metricDataPath` in the cell below) where you would like to store the metric data (this is **DIFFERENT** than path to the metric data, `outDir`). To construct metricbundles for plotting and further analysis, this path will be needed.

In [16]:
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/MetricData/'.format(your_username)

# below I am only going to run metrics on the first five opsims
for run in dbRuns[0:5]:
    # must set run name for each opSim to store metric data into
    # separate files
    maxairmassSky.setRunName(run)
    CoaddY.setRunName(run)
    CoaddU.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

Querying database SummaryAllProps with constraint filter = "g" and note not like "DD%" for columns ['airmass', 'fieldRA', 'fieldDec']
Found 211456 visits
Running:  ['maxairmassSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and note not like "DD%" for columns ['fiveSigmaDepth', 'fieldRA', 'fieldDec']
Found 146362 visits
Running:  ['CoaddU']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "y" and note not like "DD%" for columns ['fiveSigmaDepth', 'fieldRA', 'fieldDec']
Found 411654 visits
Running:  ['CoaddY']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and note not like "DD%" for columns ['airmass', 'fieldRA', 'fieldDec']
Found 198837 visits
Running:  ['maxairmassSk

Now you have computed your metric for all the opSims.  To look at those results, go through the  [View_Results.ipynb](./02_View_Results.ipynb) notebook.